## 使用python、xgboost和sklearn的随机梯度提升

用于集合决策树的简单技术涉及在训练数据集上的子样本采样。


可以采用训练数据集中的行的子集来训练成为装袋的单个树，当在计算每个分裂点时也可以获取训练数据的行的子集时，这被称为随机森林。


这些技术也可以在称为随机梯度提升的技术中用于梯度树增强模型。

## 随机梯度提升

* 梯度提升是一个贪婪的程序。
* 降薪决策树添加到模型中以校正现有模型的残差。
* 使用贪婪搜索的过程创建每个决策树，以选择最佳的最小化目标函数的分割点，这可能导致树一次又一次的使用相同的属性甚至相同的分裂点。
* Bagging是一种技术，其中创建决策树的集合，每个决策树来自训练数据的不同行的随机子集，结果时，从树集合中获得了更高的表现，因为样本中的随机性允许创建略微不同的树，从而增加了集合预测的方差。
* 通过允许在选择分割点是对特征列进行二次采样，随机森林更进一步，像书籍和添加了进一步的方差。
* 这些相同的技术可用于在称为随机梯度提升的变化中的梯度提升中的决策树的构造中。
* 通常使用训练数据的积极子样本，例如40%~80%。

## 问题描述

* 这里我们使用[Otto Group 产品分类挑战](https://www.kaggle.com/c/otto-group-product-classification-challenge)数据集。其可以从kaggle免费获得。你需要将解压狗的train.csv文件放入到你的工作目录。
* 该数据描述了超过61000种产品的93哥模糊细节，这些产品分为10哥类别（例如时装、电子产品等）。输入属性是某种不同事件的计数。
* 目标是对新产品做出预测，因为10哥类别中的每个类别都有一组概率，并且使用多类对数损失（也称交叉熵）来评价模型。
* 该竞赛在2015年完成，对于xgboost来说是一个很好的挑战，因为其有很多实例，问题的难度以及需要很少数据准备的事实（除了将字符串类型变量编码为整数）。


## 在xgboost中调整行子采样
* 行子采样涉及选择训练数据集的随机样本而无需替换。
* 行子采样可以在子样本参数中的xhboost类的sklearn包装器中指定，默认值为1.0，不进行子采样。

In [1]:
# XGBoost on Otto dataset, tune subsample
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
# load data
data = read_csv('train.csv')
dataset = data.values
# split data into X and y
X = dataset[:,0:94]
y = dataset[:,94]
# encode string class values as integers
label_encoded_y = LabelEncoder().fit_transform(y)
# grid search
model = XGBClassifier()
subsample = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
param_grid = dict(subsample=subsample)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, label_encoded_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot
pyplot.errorbar(subsample, means, yerr=stds)
pyplot.title("XGBoost subsample vs Log Loss")
pyplot.xlabel('subsample')
pyplot.ylabel('Log Loss')
pyplot.savefig('subsample.png')

d:\software\anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
